In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [18]:
# Read the csv data file as data_df
data_df = pd.read_csv("data/Billionaires Statistics Dataset.csv")
data_df.head()

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


In [19]:

# Check out the data types# Check out the data types
data_df.dtypes

# Convert gdp_country to float

# Convert finalWorth unit to billion
data_df['finalWorth'] = data_df['finalWorth']/1000

In [22]:
#fills nans in age with median age
agedata = data_df.notna()
agedata = data_df[agedata]
median_age = agedata["age"].median()
median_age
data_df['age'] = data_df['age'].fillna(value = 65)

In [23]:
#checks how many nas left in age
data_df.age.isna().sum()

0

In [79]:
# Checking rows that contain nan value
data_df[data_df.isna().any(axis=1)]

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,organization,selfMade,status,gender,birthDate,lastName,firstName,title,date,state,residenceStateRegion,birthYear,birthMonth,birthDay,cpi_country,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211.0,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,LVMH Moët Hennessy Louis Vuitton,False,U,M,3/5/1949 0:00,Arnault,Bernard,Chairman and CEO,4/4/2023 5:01,NaN,NaN,1949.0,3.0,5.0,110.05,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,6.705989e+07,46.227638,2.213749
7,8,93.0,Telecom,Carlos Slim Helu & family,83.0,Mexico,Mexico City,Telecom,Telecom,Mexico,América Móvil,True,U,M,1/28/1940 0:00,Slim Helu,Carlos,Honorary Chairman,4/4/2023 5:01,NaN,NaN,1940.0,1.0,28.0,141.54,3.6,"$1,258,286,717,125",40.2,105.8,75.0,13.1,55.1,1.260140e+08,23.634501,-102.552784
8,9,83.4,Diversified,Mukesh Ambani,65.0,India,Mumbai,Diversified,Diversified,India,Reliance Industries,False,D,M,4/19/1957 0:00,Ambani,Mukesh,Founder and Chairman,4/4/2023 5:01,NaN,NaN,1957.0,4.0,19.0,180.44,7.7,"$2,611,000,000,000",28.1,113.0,69.4,11.2,49.7,1.366418e+09,20.593684,78.962880
10,11,80.5,Fashion & Retail,Francoise Bettencourt Meyers & family,69.0,France,Paris,L'Oréal,Fashion & Retail,France,NaN,False,U,F,7/10/1953 0:00,Bettencourt Meyers,Francoise,NaN,4/4/2023 5:01,NaN,NaN,1953.0,7.0,10.0,110.05,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,6.705989e+07,46.227638,2.213749
12,13,77.3,Fashion & Retail,Amancio Ortega,87.0,Spain,La Coruna,Zara,Fashion & Retail,Spain,NaN,True,U,M,3/28/1936 0:00,Ortega,Amancio,NaN,4/4/2023 5:01,NaN,NaN,1936.0,3.0,28.0,110.96,0.7,"$1,394,116,310,769",88.9,102.7,83.3,14.2,47.0,4.707678e+07,40.463667,-3.749220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,2540,1.0,Healthcare,Yu Rong,51.0,China,Shanghai,Health clinics,Healthcare,China,NaN,True,D,M,12/14/1971 0:00,Yu,Rong,NaN,4/4/2023 5:01,NaN,NaN,1971.0,12.0,14.0,125.08,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397
2636,2540,1.0,Food & Beverage,"Richard Yuengling, Jr.",80.0,United States,Pottsville,Beer,Food & Beverage,United States,NaN,False,E,M,3/10/1943 0:00,Yuengling,Richard,NaN,4/4/2023 5:01,Pennsylvania,Northeast,1943.0,3.0,10.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
2637,2540,1.0,Manufacturing,Zhang Gongyun,60.0,China,Gaomi,Tyre manufacturing machinery,Manufacturing,China,NaN,True,R,M,12/18/1962 0:00,Zhang,Gongyun,NaN,4/4/2023 5:01,NaN,NaN,1962.0,12.0,18.0,125.08,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397
2638,2540,1.0,Real Estate,Zhang Guiping & family,71.0,China,Nanjing,Real estate,Real Estate,China,NaN,True,D,M,8/21/1951 0:00,Zhang,Guiping,NaN,4/4/2023 5:01,NaN,NaN,1951.0,8.0,21.0,125.08,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397


In [41]:
# Cleaning out the 'State' and 'Residence of State' which are not needed
full_df = data_df[['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
       'city', 'source', 'industries', 'countryOfCitizenship','selfMade', 'status', 'gender', 'birthDate', 'lastName', 'firstName',
       'title', 'date', 'birthYear',
       'birthMonth', 'birthDay', 'cpi_country', 'cpi_change_country',
       'gdp_country', 'gross_tertiary_education_enrollment',
       'gross_primary_education_enrollment_country', 'life_expectancy_country',
       'tax_revenue_country_country', 'total_tax_rate_country',
       'population_country', 'latitude_country', 'longitude_country']]

# General Informations About Billionaires Distributions

In [84]:
# Gender demographic of billionaires

# Gender proportion pie chart

# Gender with rank and wealth


In [ ]:
# Age distribution of billionaires by gender

# histogram

# boxplot to see the distribution within gender


In [ ]:
# Billionaires by industries

# Histogram

In [ ]:
# Billionaires by countries

# histogram

In [ ]:
# Geographical distribution

# Top 10 countries with most billionaires

# Use API geoapify to map the countries of billionaires


In [ ]:
# Selfmade billionaire vs. Inherited billionaire


# Correlation Analysis of Billionaires

In [ ]:
# Billionaire vs. Countries GDP


In [ ]:
# Billionaire vs. Countries Life Expectancies


In [ ]:
# Billionaires vs. Countries Population


In [ ]:
# Billionaire vs. Countries Tax Revenue


In [ ]:
# Billionaire vs. Countries CPI


In [ ]:
# Billionaire vs. Education


# Bonus: The chances of you becoming a billionaire

In [85]:
# We are only interested in selfmade billionaire for this part
# Y = billionaire/countries population %
# X = gender + country + age + industries

